In [1]:
import sys

# IT'S 2025 WHY HASN'T SOMEONE FIXED THIS
if ".." not in sys.path:
    sys.path.insert(0, "..")

import pandas as pd
import numpy as np
from tqdm.auto import tqdm

import tasks.run_helper

In [2]:
df = pd.read_json("../data/toxicity_ratings.json", lines=True)
df = df.explode(column="ratings")

ratings_df = pd.json_normalize(df.ratings)
df = pd.concat([df.reset_index(), ratings_df.reset_index()], axis=1)
df = df.drop(columns=["ratings", "index"])

df = df.loc[
    :,
    [
        "comment",
        "toxic_score",
        "personally_seen_toxic_content",
        "personally_been_target",
        "identify_as_transgender",
        "toxic_comments_problem",
        "education",
        "age_range",
        "lgbtq_status",
        "political_affilation",
        "is_parent",
        "religion_important",
    ],
]
df = df.groupby("comment").agg(list)
df["fake_index"] = 0
df = df.head(5000)
df

,toxic_score,personally_seen_toxic_content,personally_been_target,identify_as_transgender,toxic_comments_problem,education,age_range,lgbtq_status,political_affilation,is_parent,religion_important,fake_index
comment,,,,,,,,,,,,
"""Bro, can you spot me?"" &amp;#x200B; I just shit my pants a little. Did anyone get hurt?","[3, 0, 1, 4, 0]","[True, True, True, False, True]","[True, True, True, False, True]","[No, No, No, No, No]","[Rarely a problem, Very frequently a problem, ...","[Master's degree, Master's degree, Bachelor's ...","[25 - 34, 45 - 54, 45 - 54, 25 - 34, 18 - 24]","[Heterosexual, Heterosexual, Heterosexual, Het...","[Liberal, Independent, Liberal, Conservative, ...","[Yes, No, Yes, No, No]","[Somewhat important, Somewhat important, Somew...",0
"""Never"" as in ""not often, and certainly not in the recent past."" What kind of dictionary are you reading? This is what I get for never. Never adverb 1. at no time in the past or future; not ever. Slightly different from your definition. And what's wrong with targeted assassinations and overthrowing foreign governments from time to time, exactly? A lot. Its destabilising. Just look at the Middle East and see for yourself. You've only spent about 7 trillions trying to clean up your own messes. Now you are making the same mistakes in Venezuela. First you deny the US has even done these things. Now when provided ample proof of all these things, you try and brush it off as if its not a big deal. And on top of that accuse me of Gish Gallop. Are you trolling? Or just real stupid?","[0, 1, 1, 4, 3]","[False, False, True, False, True]","[False, False, True, False, True]","[No, No, No, No, No]","[Occasionally a problem, Occasionally a proble...","[Bachelor's degree in college (4-year), Master...","[25 - 34, 25 - 34, 35 - 44, 65 or older, 25 - 34]","[Heterosexual, Heterosexual, Heterosexual, Het...","[Liberal, Conservative, Independent, Liberal, ...","[Yes, Yes, No, No, Yes]","[Very important, Very important, Somewhat impo...",0
"""We are not talking about all white people","[2, 3, 0, 2, 0]","[True, True, False, True, True]","[True, True, False, True, False]","[No, No, No, Yes, No]","[Occasionally a problem, Occasionally a proble...","[Bachelor's degree in college (4-year), Bachel...","[35 - 44, 55 - 64, 55 - 64, 45 - 54, 25 - 34]","[Heterosexual, Heterosexual, Heterosexual, Het...","[Independent, Conservative, Independent, Indep...","[Yes, Yes, Yes, Yes, No]","[Somewhat important, Not important, Not import...",0
"""Yeah well if we were made in his perfect image then why would the Penis and the testicles so exposed *Points down to genitals*""","[0, 2, 1, 4, 0]","[True, True, True, False, False]","[True, False, False, False, False]","[No, No, No, No, No]","[Frequently a problem, Occasionally a problem,...","[Master's degree, Bachelor's degree in college...","[35 - 44, 18 - 24, 45 - 54, 45 - 54, 25 - 34]","[Homosexual, Heterosexual, Heterosexual, Heter...","[Liberal, Independent, Liberal, Conservative, ...","[No, No, Yes, Yes, Yes]","[Not important, Somewhat important, Not import...",0
"""i'm offended"" - tell them that you're offended by their obnoxious behavior","[0, 1, 0, 0, 0]","[True, False, True, False, True]","[False, False, True, False, False]","[No, No, No, No, No]","[Occasionally a problem, Occasionally a proble...",[High school graduate (high school diploma or ...,"[25 - 34, 25 - 34, 25 - 34, 35 - 44, 45 - 54]","[Heterosexual, Heterosexual, Heterosexual, Het...","[Independent, Liberal, Independent, Liberal, C...","[No, Yes, Yes, No, Yes]","[Very important, Somewhat important, Very impo...",0
...,...,...,...,...,...,...,...,...,...,...,...,...
"AND SHE WILL WIN. NO OTHER WAY TO LOOK AT IT. IT'S TIME FOR 24/7 COVERAGE OF WHAT MITCH HAS COST THIS NATION WITH HIS ONE TRACK AGENDA. HOLDING UP EVERYTHING, IN EFFECT THROWING ALL OUR TAX MONEY AWAY. HE IS DECIDING FOR 300 MILLION PEOPLE HOW TO RUN THIER COUNTRY. BAD BAD BAD...","[1, 1, 0, 1, 0]","[False, True, True, True, True]","[F

In [3]:
sdb_columns = [
    "personally_seen_toxic_content",
    "personally_been_target",
    "identify_as_transgender",
    "toxic_comments_problem",
    "education",
    "age_range",
    "lgbtq_status",
    "political_affilation",
    "is_parent",
    "religion_important",
]


for sdb_column in tqdm(sdb_columns):
    print("*" * 10)
    print(sdb_column)
    print(
        tasks.run_helper.results(
            df,
            discussion_id_col="fake_index",
            sdb_column=sdb_column,
            value_col="toxic_score",
            comment_key_col="comment",
        )
    )

  0%|          | 0/10 [00:00<?, ?it/s]

**********
personally_seen_toxic_content
              0          
          value    pvalue
False  0.026291  0.313526
True  -0.039500  0.538900
**********
personally_been_target
              0          
          value    pvalue
False -0.048334  0.551588
True   0.030185  0.358057
**********
identify_as_transgender
                          0          
                      value    pvalue
Yes               -0.000575  0.040613
Prefer not to say  0.001355  0.028219
No                -0.008726  0.449004
**********
toxic_comments_problem
                                  0          
                              value    pvalue
Not a problem              0.011777  0.097828
Very frequently a problem -0.005873  0.133509
Occasionally a problem    -0.003724  0.408576
Rarely a problem           0.006905  0.233949
Frequently a problem      -0.015699  0.328759
**********
education
                                                           0          
                                            

In [ ]:
df["random"] = df.religion_important.apply(
    lambda x: [np.random.randint(1, 10) for _ in range(len(x))]
)
tasks.run_helper.results(
    df,
    discussion_id_col="fake_index",
    sdb_column="random",
    value_col="toxic_score",
    comment_key_col="comment",
)